In [8]:
# Populate the data in resources for all counties
# Dependencies
import requests
from dotenv import load_dotenv
import os
import pandas as pd
from bs4 import BeautifulSoup
import scipy.stats as st
import matplotlib.pyplot as plt
import subprocess
import time
from requests.exceptions import ConnectTimeout, ReadTimeout, RequestException

In [9]:
# Set environment variables from the .env in the local environment
def key_check(key_path=None):
    try:
        reply=load_dotenv(key_path,verbose=True,override=True)   
        assert reply , 'Dotenv is not found'
        fred_api_key = os.getenv("FRED_API_KEY")
        assert fred_api_key is not None, 'FRED_API_KEY not found in .env file'
        responce=requests.get(f'https://api.stlouisfed.org/fred/category/series?category_id=125&api_key={fred_api_key}')
        assert responce.status_code == 200, f'The key provided failed to authenticate tmdb_api_key {fred_api_key} code {responce.status_code}'
    except Exception as e:
        # Handle potential errors in loading .env or missing API keys
        print(f'An error occurred: {e}')
        return(False)
    else:
        print('All keys loaded correctly')
        return (True)



In [10]:
def run_notebook(path):
    """Run a Jupyter Notebook via nbconvert and capture its output."""
    cmd = ['jupyter', 'nbconvert', '--to', 'notebook', '--execute',
           '--output', path, path]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result

In [11]:
# read the file into a list
def load_series_id():
    # retry variables
    max_attempts = 3
    attempts = 0
    # List of notebook paths
    notebooks = ['c:/SRC/AI/ProjectOne_Team2/Frank_playground/Fred_get_HPI.ipynb',
                'c:/SRC/AI/ProjectOne_Team2/Frank_playground/Fred_get_HHI.ipynb',
                'c:/SRC/AI/ProjectOne_Team2/Frank_playground/Fred_get_POP.ipynb']
    # Start load Series_id_df
    while attempts < max_attempts:
        try:
            hpi_series_df = pd.read_csv('C:\\SRC\\AI\\ProjectOne_Team2\\resources\\TX_County_HPI_ID.csv',index_col='County')
            hhi_series_df = pd.read_csv('C:\\SRC\\AI\\ProjectOne_Team2\\resources\\TX_County_HHI_ID.csv',index_col='County')
            pop_series_df = pd.read_csv('C:\\SRC\\AI\\ProjectOne_Team2\\resources\\TX_County_POP_ID.csv',index_col='County')
            final_merged_df = hpi_series_df.join([hhi_series_df, pop_series_df], how='inner')
            display (final_merged_df.head(3))
            print("Files loaded successfully.")
            return final_merged_df
        except FileNotFoundError:
            print(f"Attempt {attempts + 1} failed. Trying to regenerate files.")
            # Start each notebook in a separate process
            processes = [subprocess.Popen(['jupyter', 'nbconvert', '--to', 'notebook', '--execute',
                '--output', nb, nb], stdout=subprocess.PIPE, stderr=subprocess.PIPE) for nb in notebooks] 
            # Wait for all processes to complete
            for proc in processes:
                proc.wait()
            # Increment the attempt counter
            attempts += 1


In [12]:
# this section of code make a request and permits up to max_retries 
def get_page(url, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            # Attempt to make the HTTP GET request with a specified timeout
            response = requests.get(url, timeout=2)  # adding a sensible timeout
            # If the request is successful, break out of the loop
            if response.status_code == 200:
                return response.json()
            if response.status_code == 404:
                return None , None
        except (ConnectTimeout, ReadTimeout) as e:
            # Print an error message showing the retry attempt
            print(f"Attempt {retries + 1} of {max_retries} failed with timeout. Retrying...")
            retries += 1
            if retries == max_retries:
                print("Max retries exceeded. Failing...")
                raise  # Re-raise the last exception after final attempt
        except RequestException as e:
            print(f"Attempt {retries + 1} failed with a request exception: {e}. Retrying...")
            retries += 1
            if retries == max_retries:
                print("Max retries exceeded with request exceptions. Failing...")
                raise
        except: 
            return None, None

    # If all retries are exhausted without a return, it indicates a failure
    print("Request failed after maximum retries.")
    return None, None


In [20]:
#  Set variable for all cells

rt_start='&observation_start=1997-01-01'
rt_end='&observation_end=2022-01-01'
# series_id=''
# url = (f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}{rt_start}{rt_end}&api_key={fred_api_key}&file_type=json")
# series=[]
series_corr_list=[]


In [51]:
# get API keys
my_env_path='C:\SRC\AI\ProjectOne_Team2\Frank_playground\keys.env'
if key_check(my_env_path):
    fred_api_key = os.getenv("FRED_API_KEY")
else:
    print ('fix Keys and rerun')
    
# Load Series ID by county for HPI, HHI and POP
county_series_ids=load_series_id()
series_hpi = []
series_hhi = []
series_pop = []

# retrieve data by county for HPI, HHI and POP for date rage set in VAR
for index, row in county_series_ids.iterrows():
    # Get HPI
    series_id=county_series_ids.loc[index,'HPI ID']
    url = (f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}{rt_start}{rt_end}&api_key={fred_api_key}&file_type=json")
    series = get_page(url)

    for item in series["observations"]:
        if item['value'] != '.':
                series_hpi.append({'county' : index,'date': item['date'], 'value': item['value']})      
    
    # get HHI
    series_id=county_series_ids.loc[index,'HHI ID']
    url = (f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}{rt_start}{rt_end}&api_key={fred_api_key}&file_type=json")
    series = get_page(url)

    for item in series["observations"]:
        if item['value'] != '.':
                series_hhi.append({'county' : index,'date': item['date'], 'value': item['value']})

    # get POP
    series_id=county_series_ids.loc[index,'POP ID']
    url = (f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}{rt_start}{rt_end}&api_key={fred_api_key}&file_type=json")
    series = get_page(url)

    for item in series["observations"]:
        if item['value'] != '.':
                series_pop.append({'county' : index,'date': item['date'], 'value': item['value']})
 

All keys loaded correctly


,HPI ID,HHI ID,POP ID
County,,,
Anderson County,ATNHPIUS48001A,MHITX48001A052NCEN,TXANDE1POP
Andrews County,ATNHPIUS48003A,MHITX48003A052NCEN,TXANDR3POP
Angelina County,ATNHPIUS48005A,MHITX48005A052NCEN,TXANGE5POP


Files loaded successfully.


In [58]:
   
hpi_df = pd.DataFrame(series_hpi)
hpi_df = (
         hpi_df
        .assign(date=lambda df: pd.to_datetime(df['date']))  # Convert 'date' to datetime
        .rename(columns={'value': 'HPI'})   # Rename 'value' column to 'HPI'
        .set_index(['county', 'date'])                  # Set 'date' as the index
        .dropna()                           # Drop rows with NaN values
        )
display (hpi_df.head(3))
hhi_df = pd.DataFrame(series_hhi)
hhi_df = (
         hhi_df
        .assign(date=lambda df: pd.to_datetime(df['date']))  # Convert 'date' to datetime
        .rename(columns={'value': 'HHI'})   # Rename 'value' column to 'HPI'
        .set_index(['county', 'date'])                    # Set 'date' as the index
        .dropna()                           # Drop rows with NaN values
        )
pop_df = pd.DataFrame(series_pop)
pop_df = (
         pop_df
        .assign(date=lambda df: pd.to_datetime(df['date']))     # Convert 'date' to datetime
        .rename(columns={'value': 'pop'})                       # Rename 'value' column to 'HPI'
        .set_index(['county', 'date'])                          # Set 'date' as the index
        .sort_index(ascending=[False,True])
        #.dropna()                                               # Drop rows with NaN values
        )
series_id_df= hpi_df.join(hhi_df, how='inner').join(pop_df, how='inner')
display (series_id_df.head(5))
display (series_id_df.tail(5))

HPI
county          date             
Anderson County 1997-01-01  91.01
                1998-01-01  92.84
                1999-01-01  97.80

HPI    HHI     pop
county          date                             
Anderson County 1997-01-01   91.01  29760  54.051
                1998-01-01   92.84  30638  54.290
                1999-01-01   97.80  30385  54.876
                2000-01-01  100.00  31930  55.061
                2001-01-01  104.39  31113  54.127

HPI    HHI     pop
county       date                             
Young County 2018-01-01  165.81  49301  17.953
             2019-01-01  171.94  52643  17.937
             2020-01-01  175.16  54569  17.849
             2021-01-01  200.74  53715  17.943
             2022-01-01  247.64  58297  18.012

In [19]:
  
texas_counties_df=pd.DataFrame(series_list)
texas_counties_df.head(3)
    
    
'''
    pop_df = (
        pop_df
        .assign(date=lambda df: pd.to_datetime(df['date']))  # Convert 'date' to datetime
        .rename(columns={'value': 'POP'})   # Rename 'value' column to 'HPI'
        .set_index('date')                  # Set 'date' as the index
        .dropna()                           # Drop rows with NaN values
        )
    
    
    series_df= series_df.join(pop_df, how='inner') 
    # display (series_df.info())
    # display (series_df)
    series_df['HPI'] = series_df['HPI'].astype(float)
    series_df['HHI'] = series_df['HHI'].astype(float)
    series_df['POP'] = series_df['POP'].astype(float)

    
    ## get the hpi/hhi coorilation
    hpi = series_df.loc[:,'HPI']
    hhi = series_df.loc[:,'HHI']
    hpi_hhi_corr = st.pearsonr(hpi,hhi)


    # display (f"{index} HPI/Income correlation  is {(round(hpi_hhi_corr[0]*100,1))}%")

    ## get the hpi/hhi coorilation
    hpi = series_df.loc[:,'HPI']
    pop = series_df.loc[:,'POP']
    hpi_pop_corr = st.pearsonr(hpi,pop)
    display (hpi_pop_corr)
    series_corr_list.append([index,round(hpi_hhi_corr[0]*100),round(hpi_pop_corr[0]*100)])
    # display (f"{index} HPI/HPI {(round(hpi_hhi_corr[0]*100,1))}% HPI/POP {(round(hpi_pop_corr[0]*100,1))}%")
    # CREATE A FILENAME AND WRITE THE DATA TO THE FILE.
    # filename= (f'{index.replace(" ","")}_HPI_HHI_POP.CSV')
    # file_path = f"../resources/{filename}"  # Construct file path with .csv extension
    # series_df.to_csv(file_path, index=True)
'''

'\n    pop_df = (\n        pop_df\n        .assign(date=lambda df: pd.to_datetime(df[\'date\']))  # Convert \'date\' to datetime\n        .rename(columns={\'value\': \'POP\'})   # Rename \'value\' column to \'HPI\'\n        .set_index(\'date\')                  # Set \'date\' as the index\n        .dropna()                           # Drop rows with NaN values\n        )\n    \n    \n    series_df= series_df.join(pop_df, how=\'inner\') \n    # display (series_df.info())\n    # display (series_df)\n    series_df[\'HPI\'] = series_df[\'HPI\'].astype(float)\n    series_df[\'HHI\'] = series_df[\'HHI\'].astype(float)\n    series_df[\'POP\'] = series_df[\'POP\'].astype(float)\n    # df[\'HHI\'] = df[\'HHI\'].astype(float)\n    # df[\'POP\'] = df[\'POP\'].astype(float)\n    ## get the hpi/hhi coorilation\n    hpi = series_df.loc[:,\'HPI\']\n    hhi = series_df.loc[:,\'HHI\']\n    hpi_hhi_corr = st.pearsonr(hpi,hhi)\n\n\n    # display (f"{index} HPI/Income correlation  is {(round(hpi_hhi_corr[

In [ ]:
Corr_df = pd.DataFrame(series_corr_list, columns=['County', 'HPI/HHI', 'HPI/POP'])
Corr_df.set_index('County',inplace=True)
display (Corr_df.head(3))

hpi_hhi_counts = Corr_df['HPI/HHI'].value_counts().sort_index()
hpi_pop_counts = Corr_df['HPI/POP'].value_counts().sort_index()

# Plotting the frequency of HPI/HHI
plt.figure(figsize=(16, 5))
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
hpi_hhi_counts.plot(kind='bar')
plt.title('Frequency of HPI/HHI Values')
plt.xlabel('HPI/HHI')
plt.ylabel('Frequency')
plt.show()

# Plotting the frequency of HPI/POP
plt.figure(figsize=(16, 5))
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
hpi_pop_counts.plot(kind='bar')
plt.title('Frequency of HPI/POP Values')
plt.xlabel('HPI/POP')
plt.ylabel('Frequency')

#plt.tight_layout()
plt.show()